# ART for TensorFlow v2 - Keras API

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
from matplotlib import pyplot as plt

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod, CarliniLInfMethod

# Load RML2016 

In [2]:
# Import all the things we need ---
#   by setting env variables before Keras import you can set up which backend and which GPU it uses
import os,random
import keras
#os.environ["KERAS_BACKEND"] = "theano"

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["CUDA_VISIBLE_DEVICE"]  = '0'
#os.environ["THEANO_FLAGS"]  = "floatX=float32"
#os.environ["THEANO_FLAGS"]  = "device=cuda%d"%(1)

Using TensorFlow backend.


In [3]:
import numpy as np
import seaborn as sns
import pickle, random, sys

In [4]:
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
#from keras.optimizers import adam
from tensorflow.keras.optimizers import Adam
#import theano as th
#import theano.tensor as T
import os
WEIGHTS_PATH = ('resnet_like_weights_tf_dim_ordering_tf_kernels.h5')
from keras.models import Model
from keras.layers import Input,Dense,Conv1D,MaxPool1D,ReLU,Dropout,Softmax
from keras.layers import LSTM
import keras
from keras.callbacks import LearningRateScheduler
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from keras.models import model_from_json
import tensorflow as tf

In [5]:
import importlib,sys

importlib.reload(sys)

<module 'sys' (built-in)>

In [6]:
from mpl_toolkits.mplot3d import Axes3D

from sklearn.preprocessing import StandardScaler

In [7]:
# Load the dataset ...
#  You will need to seperately download or generate this file
dbfile = open('RML2016.10a_dict.dat', 'rb')      
Xd = pickle.load(dbfile,encoding='latin1') 

In [8]:
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

In [9]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = int(n_examples * 0.9)
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [10]:
print('数据集总数：',n_examples)
print('调制方式' , len(mods),'种:' ,mods)
print('信噪比:',snrs)

数据集总数： 220000
调制方式 11 种: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']
信噪比: [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [156]:
# AP
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

In [157]:
def AP(signal):
    x = signal[1]/signal[0]
    X_p = np.arctan(x)
    X_a = (signal[0]**2 + signal[1]**2)**0.5
    signal = np.stack((X_a,X_p), axis=0)
    return signal

In [158]:
def trans_to_AP(signal_set):
    for i in range(signal_set.shape[0]):
        signal_set[i] = AP(signal_set[i])
    return signal_set

In [159]:
X_train_AP = trans_to_AP(X_train_copy)
X_test_AP = trans_to_AP(X_test_copy)
X_train_AP.shape,X_test.shape

((198000, 2, 128), (22000, 2, 128))

In [160]:
X_train_AP=X_train_AP.swapaxes(2,1)
X_test_AP = X_test_AP.swapaxes(2,1)

X_train_AP.shape,X_test_AP.shape

((198000, 128, 2), (22000, 128, 2))

In [161]:
X_train = X_train.swapaxes(2,1)
X_test = X_test.swapaxes(2,1)

X_train.shape,X_test.shape

((198000, 128, 2), (22000, 128, 2))

In [162]:
X_test = X_test.swapaxes(2,1)
X_test.shape

(22000, 2, 128)

## 指标定义

In [163]:
# PNR
# PSR
def PSR(PSR_db):
    return 10**(PSR_db/10)
def PSR_db(PSR):
    return 10*np.log10(PSR)
def PNR(PSR_db,snr):
    return PSR_db + snr


def compute_power(signal):
    '''
    计算扰动功率
    '''      
    #siganl_re = signal.reshape(-1,256)
    SigPow = (sum(np.linalg.norm(signal.reshape([-1,256]),axis=1))**2/signal.shape[0])
    #print('sig',np.sum(np.linalg.norm(signal.reshape([-1,256]),axis=1)))
    #SigPow = np.linalg.norm(signal.reshape([-1,256]))
    return SigPow
    
#def compute_norm(signal):
 #   return np.sum(np.linalg.norm(signal.reshape([-1,256]),axis=1))

## Attck models

In [142]:
# 加载模型
from tensorflow.keras.models import load_model
model_cnn2 = load_model('C:\\Users\\Serendipity\\AMC_Feature_Representation\\model_saved\\Sequence_based\\VTCNN2_AP.h5')
model_cldnn = load_model('C:\\Users\\Serendipity\\AMC_Feature_Representation\\model_saved\\Sequence_based\\CLDNN_AP.h5')
model_resnet = load_model('C:\\Users\\Serendipity\\AMC_Feature_Representation\\model_saved\\Sequence_based\\ResNet_AP.h5')
model_vgg = load_model('C:\\Users\\Serendipity\\AMC_Feature_Representation\\model_saved\\Sequence_based\\VGG_AP.h5')

In [143]:
model_cnn2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1, 128, 2)         0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 1, 132, 2)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 132, 256)       3328      
_________________________________________________________________
dropout (Dropout)            (None, 1, 132, 256)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 1, 136, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 136, 80)        122960    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 136, 80)        0

In [144]:
# IQ_fgsm
def eps_cal(PNR):
    return 0.002*np.exp(0.1152*PNR)

#IQ_pgd
# def eps_cal(PNR):
#     return 0.0008*np.exp(0.1165*PNR)

#IQ_uap
# def eps_cal(PNR):
#     return 0.001*np.exp(0.1358*PNR)

# AP
# def eps_cal(PNR):
#     return 0.0811*np.exp(0.1151*PNR)

In [48]:
PSR = 0.017136584068757895
#print('PNR_cal = ',PNR(PSR_db(PSR),snr))

In [52]:
'asda' + '_upbound_' + str(PSR)

'asda_upbound_0.017136584068757895'

In [113]:
# 总体评价
classes = mods


#X_test = X_test_AP

def attck_pnr(model,snr = 10):
    '''
    对单个模型构建白盒攻击
    输出：SNR=10dB时,PNR在-30~10之间的攻击结果
    '''
    att_acc = []
    #①转为ART类
    classifier = KerasClassifier(model=model)
    for pnr in range(-30,10):
        EPS = eps_cal(pnr)
        #②定义攻击算法
        #attack_fgsm = FastGradientMethod(estimator=classifier, eps=EPS)
        #attack_pgd = ProjectedGradientDescent(estimator = classifier, eps=EPS,max_iter=10)
        attack_uap = UniversalPerturbation(classifier = classifier, eps = EPS, attacker = 'fgsm')
        #③生成对抗样本
        #x_test_adv = attack_fgsm.generate(X_test)
        x_test_adv = attack_pgd.generate(X_test)
        
        
        #④验证pnr
        PSR = (compute_power(x_test_adv-X_test)/compute_power(X_test))
        print('PNR_cal = ',PNR(PSR_db(PSR),snr))
        print('PNR_set = ',pnr)
        #⑤ 求acc
        test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
        test_X_i = x_test_adv[np.where(np.array(test_SNRs)==snr)]
        test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   
        #print(test_X_i)
        test_Y_i_hat = model.predict(test_X_i)
        # estimate classes
        test_Y_i_hat = model.predict(test_X_i)
        conf = np.zeros([len(classes),len(classes)])
        confnorm = np.zeros([len(classes),len(classes)])
        for i in range(0,test_X_i.shape[0]):
            j = list(test_Y_i[i,:]).index(1)
            k = int(np.argmax(test_Y_i_hat[i,:]))
            conf[j,k] = conf[j,k] + 1
#         for i in range(0,len(classes)):
#             confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
        #plt.figure()
        #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))

        cor = np.sum(np.diag(conf))
        ncor = np.sum(conf) - cor
        #print("Overall Accuracy: ", cor / (cor+ncor))
        acc = 1.0*cor/(cor+ncor)
        print(acc)
        att_acc.append(acc)
#         str = "pnr = %d acc = %0.2f" %(PNR(PSR_db(PSR),snr),acc)
#         print(str)
    return att_acc

# BLACK BOX

In [117]:
# ① 提取SNR = 10dB的测试样本
def extract_snr_10dB(snr = 10):
    test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
    test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]
    return test_X_i, test_Y_i

# ② 设置PNR，生成基于代理模型的对抗样本
classes = mods

def generate_AEs(model,pnr,attack_samples,snr = 10):
    '''
    基于代理模型，构建对抗扰动
    输出：基于代理模型的对抗样本
    '''
    EPS = eps_cal(pnr)
    
    classifier = KerasClassifier(model=model)
    #②定义攻击算法
    attack_fgsm = FastGradientMethod(estimator=classifier, eps=EPS)
    #attack_pgd = ProjectedGradientDescent(estimator=classifier, eps=EPS,max_iter=20)
    #attack_uap = UniversalPerturbation(classifier = classifier, eps = EPS, attacker = 'fgsm')
    
    #③生成对抗样本 snr = 10dB
    x_test_adv = attack_fgsm.generate(attack_samples)
    #x_test_adv = attack_pgd.generate(attack_samples)
    #x_test_adv = attack_uap.generate(attack_samples)
    
    return x_test_adv,EPS

# ③模型黑盒攻击，输出SNR=10dB各模型攻击结果
def attack_results(test_X_i,test_Y_i,model):
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
#         for i in range(0,len(classes)):
#             confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    #plt.figure()
    #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    #print("Overall Accuracy: ", cor / (cor+ncor))
    acc = 1.0*cor/(cor+ncor)
    return acc
    #print(acc)

In [118]:
# test ae
from art.attacks.evasion import ProjectedGradientDescent, UniversalPerturbation
# ① 提取SNR = 10dB的测试样本
test_X_10dB, test_Y_10dB = extract_snr_10dB(snr = 10)

In [119]:
# ③ 模型黑盒攻击，输出SNR=10dB各模型攻击结果
model_cldnn_acc = []
model_vgg_acc = []
model_cnn2_acc = []
model_resnet_acc = []
EPS_CAL = []
PNR_CAL = []
for pnr in range(-30,10): 
    x_test_adv,EPS = generate_AEs(model_cnn2,pnr=pnr,attack_samples = test_X_10dB)
    #④验证pnr
    PSR = (compute_power(x_test_adv-test_X_10dB)/compute_power(test_X_10dB))
    PNR_CAL.append(PNR(PSR_db(PSR),10))
    EPS_CAL.append(EPS)
    model_cldnn_acc.append(attack_results(x_test_adv,test_Y_10dB,model_cldnn))
    model_vgg_acc.append(attack_results(x_test_adv,test_Y_10dB,model_vgg))
    model_cnn2_acc.append(attack_results(x_test_adv,test_Y_10dB,model_cnn2))
    model_resnet_acc.append(attack_results(x_test_adv,test_Y_10dB,model_resnet))

In [120]:
import pandas as pd
writer = pd.ExcelWriter('ATT_FGSM_IQ.xlsx')		# 写入Excel文件

pd.DataFrame(PNR_CAL).to_excel(writer, 'PNR_CAL', float_format='%.5f')
pd.DataFrame(EPS_CAL).to_excel(writer, 'EPS_CAL', float_format='%.5f')
pd.DataFrame(model_cldnn_acc).to_excel(writer, 'model_cldnn_acc', float_format='%.5f')
pd.DataFrame(model_cnn2_acc).to_excel(writer, 'model_cnn2_acc', float_format='%.5f')
pd.DataFrame(model_vgg_acc).to_excel(writer, 'model_vgg_acc', float_format='%.5f')
pd.DataFrame(model_resnet_acc).to_excel(writer, 'model_resnet_acc', float_format='%.5f')
writer.save()

writer.close()

In [87]:
pnr = -30
#EPS = eps_cal(pnr)
EPS = 0.001

In [90]:
# test ae
from art.attacks.evasion import ProjectedGradientDescent, UniversalPerturbation,CarliniL2Method,CarliniWagnerASR

classifier = KerasClassifier(model=model_cnn2)

attack_fgsm = FastGradientMethod(estimator=classifier, eps=EPS)
attack_pgd = ProjectedGradientDescent(estimator = classifier, eps = EPS, max_iter=10)
attack_uap = UniversalPerturbation(classifier = classifier, eps = EPS, attacker = 'fgsm')
attack_cw = CarliniL2Method(classifier = classifier,confidence=10)

In [50]:
test_X_10dB = test_X_10dB.swapaxes(2,1)
test_X_10dB.shape

(1066, 128, 2)

In [56]:
x_test_adv = attack_cw.generate(test_X_10dB)

C&W L_2:   0%|          | 0/1066 [00:00<?, ?it/s]

In [61]:
for i in range(20,60,10):
    print(i)

20
30
40
50


In [57]:
PSR = (compute_power(x_test_adv-test_X_10dB)/compute_power(test_X_10dB))
print('PNR_cal = ',PNR(PSR_db(PSR),snr))
print('PNR_set = ',pnr)

PNR_cal =  -inf
PNR_set =  -30


In [58]:
attack_results(x_test_adv,test_Y_10dB,model_cldnn),attack_results(x_test_adv,test_Y_10dB,model_vgg),attack_results(x_test_adv,test_Y_10dB,model_cnn2),attack_results(x_test_adv,test_Y_10dB,model_resnet)

(0.8236397748592871,
 0.8358348968105066,
 0.7551594746716698,
 0.8208255159474672)

In [62]:
for i in range(20,21):
    print(i)

20


In [160]:
# model0:原模型测试
classes = mods
acc = {}

test_X_i = x_test_adv
test_Y_i_hat = model.predict(test_X_i)
# estimate classes
test_Y_i_hat = model.predict(test_X_i)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for i in range(0,test_X_i.shape[0]):
    j = list(test_Y_10dB[i,:]).index(1)
    k = int(np.argmax(test_Y_i_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
#plt.figure()
#plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))

cor = np.sum(np.diag(conf))
ncor = np.sum(conf) - cor
#print("Overall Accuracy: ", cor / (cor+ncor))
print(cor / (cor+ncor))
acc[snr] = 1.0*cor/(cor+ncor)

0.0947467166979362


In [ ]:
import pandas as pd
writer = pd.ExcelWriter('att.xlsx')		# 写入Excel文件
pd.DataFrame(attck_results).to_excel(writer, 'att_iq_pgd', float_format='%.5f')		# ‘page_1’是写入excel的sheet名
writer.save()

writer.close()

In [48]:
# #loss_test, accuracy_test = model.evaluate(x_test_adv, Y_test)
# perturbation = np.mean(np.abs((x_test_adv - X_test)))
# #print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
# print('Average perturbation: {:4.2f}'.format(perturbation))

Average perturbation: 0.14
